In [1]:
import pandas as pd
import MeCab as mc
from wordcloud import WordCloud
import os
import re

# Twitterの外部ライブラリ(GOT3)の使い方
python3 cli.py -s (検索ワード) --since "2022-12-05"(検索開始日時) --untill "2022-12-06"(検索終了日時) -o user.csv(ファイル名) --csv

In [19]:
def reduceTwitterData(rawData):
    df = pd.read_csv("../GetOldTweets3-0.0.10/" + rawData)
    print(df)
    df.to_csv('./csv_Data/revised_'+ rawData, columns = ['date', 'time', 'timezone', 'tweet'], index = False)

In [3]:
def strip_CRLF_from_Text(text):
    """
    改行前後が日本語文字の場合は改行を削除する．
    それ以外はスペースに置換する．
    """
    # 改行前後の文字が日本語文字の場合は改行を削除する
    plaintext = re.sub('([ぁ-んー]+|[ァ-ンー]+|[\\u4e00-\\u9FFF]+|[ぁ-んァ-ンー\\u4e00-\\u9FFF]+)(\n)([ぁ-んー]+|[ァ-ンー]+|[\\u4e00-\\u9FFF]+|[ぁ-んァ-ンー\\u4e00-\\u9FFF]+)',
                       r'\1\3',
                       text)
    plaintext = re.sub('http(s)?://([\w]+\.)+[\w]+(/[\w ./?%&=]*)?', "", plaintext)
    # 残った改行とタブ記号はスペースに置換する
    plaintext = plaintext.replace('\n', ' ').replace('\t', ' ')
    return plaintext

In [4]:
def mecab_wakati(text):
    #t = mc.Tagger()
    t = mc.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
    node = t.parseToNode(text)
    #print(type(node))
    sent = ""
    while(node):
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            # 名詞だけをリストに追加する
            # if word_type in ["名詞"] and (("日本") not in node.surface) and (("代表") not in node.surface) and (("選手") not in node.surface) and (("監督") not in node.surface) and (("試合") not in node.surface):
            #      sent += node.surface + " "  # node.surface は「表層形」
            # 動詞（の原型），形容詞，副詞もリストに加えたい場合は次の２行を有効にする
            if word_type in [ "形容動詞", "形容詞"]:
                sent += node.feature.split(",")[6] + " " 
        node = node.next
        if node is None:
            break
    return sent

In [18]:
def twitterdataToWordCloud(CSV_FILE):
    df = pd.read_csv("./csv_data/revised_" + CSV_FILE)
    tweet = df.loc[: , "tweet"]
    tweetcontent = ""
    for i in range(len(tweet)):
        tweetcontent += tweet[i]
    # f = open(tweetcontent, encoding="utf8")
    # raw = f.read()
    #print(raw)
    text = strip_CRLF_from_Text(tweetcontent)
    #print(text)
    sent = mecab_wakati(text)
    sent = re.sub('(\s)+(.)(\s)+', ' ', sent)
    # フォントの保存先を指定する
    font_path = "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"  ## Mac 版はこちら
    wc = WordCloud(max_font_size=36, font_path=font_path).generate(sent)
    wc.to_file("./wordCloud/wc-" + CSV_FILE[:-3] + "png")

In [14]:
def main(rawdata):  #nation.csv
    reduceTwitterData(rawdata)
    twitterdataToWordCloud(rawdata)

In [25]:
main("イングランド-2022.csv")

                       id      conversation_id               created_at  \
0     1603384768611098624  1603384768611098624  2022-12-15 22:41:32 JST   
1     1603384725070315520  1602707206239289344  2022-12-15 22:41:22 JST   
2     1603383000569622528  1603336876257005569  2022-12-15 22:34:31 JST   
3     1603381095969558530  1603381095969558530  2022-12-15 22:26:57 JST   
4     1603380654380503040  1603380654380503040  2022-12-15 22:25:12 JST   
...                   ...                  ...                      ...   
4588  1598848498790985728  1598848498790985728  2022-12-03 10:16:01 JST   
4589  1598848153700413442  1598848153700413442  2022-12-03 10:14:39 JST   
4590  1598842558498902016  1598842356509585408  2022-12-03 09:52:25 JST   
4591  1598839335364997120  1598839335364997120  2022-12-03 09:39:37 JST   
4592  1598830687280193536  1598830687280193536  2022-12-03 09:05:15 JST   

            date      time  timezone              user_id         username  \
0     2022-12-15  22: